# Todo
1. zero mean and var

In [2]:
import src.dataset as ds
import numpy as np
from src.embeddings import extract_embedding_weights
from keras.layers import Embedding, CuDNNLSTM, Bidirectional, Dense
from keras.initializers import Constant
from keras import Sequential
from tensorflow.python.keras.callbacks import TensorBoard
from time import time
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

In [3]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

In [4]:
X, y, tokenizer = ds.load_tokenized_data()
embedding_matrix = extract_embedding_weights()

100%|██████████████████████████████████████████████████████| 615/615 [00:01<00:00, 311.28it/s]


In [5]:
def make_model():
    # load pre-trained word embeddings into an Embedding layer
    # note that we set trainable = False so as to keep the embeddings fixed
    EMBEDDING_DIM = 300
    num_words = len(tokenizer.word_index) + 1
    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                embeddings_initializer=Constant(embedding_matrix),
                                input_length=1,
                                trainable=False)
    # define model
    model = Sequential()
    model.add(embedding_layer)
    model.add(Bidirectional(CuDNNLSTM(50)))
    model.add(Dense(num_words, activation='softmax'))
    model.summary()
    return model
    
    
def train(model, epochs=5):
    # compile network
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(X, y, epochs=epochs, verbose=2, callbacks=[tensorboard])
    
    
# generate a sequence from the model
def generate_seq(model, tokenizer, seed_text, n_words):
	in_text, result = seed_text, seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		encoded = np.array(encoded)
		# predict a word in the vocabulary
		yhat = model.predict_classes(encoded, verbose=0)
    		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text, result = out_word, result + ' ' + out_word
	return result

In [6]:
model = make_model()

W0721 21:34:59.468410  7768 deprecation_wrapper.py:119] From C:\Users\Eli\Anaconda3\envs\dlinto\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0721 21:34:59.482373  7768 deprecation_wrapper.py:119] From C:\Users\Eli\Anaconda3\envs\dlinto\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0721 21:34:59.929178  7768 deprecation_wrapper.py:119] From C:\Users\Eli\Anaconda3\envs\dlinto\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 300)            2251800   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100)               140800    
_________________________________________________________________
dense_1 (Dense)              (None, 7506)              758106    
Total params: 3,150,706
Trainable params: 898,906
Non-trainable params: 2,251,800
_________________________________________________________________


In [8]:
train(model)

AttributeError: 'Sequential' object has no attribute 'run_eagerly'

In [ ]:
generate_seq(model, tokenizer, 'i', 10)